# load model


# cav-mae 的整体模型定义并导入预训练权重

In [2]:
import os
import torch
import timm
from src.models import CAVMAE

assert timm.__version__ == '0.4.5' # it is important to have right version of timm

model_path = 'pretrained_model/audio_model.pth'
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7" # select GPU
# CAV-MAE model with decoder
#print(torch.cuda.device_count())
torch.cuda.set_device(1)
audio_model = CAVMAE(audio_length=1024,
                     modality_specific_depth=11,
                     norm_pix_loss=True, tr_pos=False) # most models are trained with pixel normalization and non-trainabe positional embedding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mdl_weight = torch.load(model_path, map_location=device)
audio_model = torch.nn.DataParallel(audio_model) # it is important to convert the model to dataparallel object as all weights are saved in dataparallel format (i.e., in module.xxx)
miss, unexpected = audio_model.load_state_dict(mdl_weight, strict=False)
print('miss=',miss, 'unexpected=', unexpected) # check if all weights are correctly loaded

A CAV-MAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
Number of Audio Patches: 512, Visual Patches: 196
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 196, 768])
miss= [] unexpected= []


# 定义cav-mae模型，查看其中的video模块输出维度

In [3]:
cav_mae_model = CAVMAE(audio_length=1024,
                       modality_specific_depth=11,
                       norm_pix_loss=True, tr_pos=False)

A CAV-MAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
Number of Audio Patches: 512, Visual Patches: 196
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 196, 768])


In [4]:
audio_data = torch.rand(1, 1024, 128)
video_data = torch.rand(1, 3, 224, 224)
output = cav_mae_model(audio_data, video_data)

NotImplementedError: 

# 定义mae dfer模型

In [ ]:
from src.models import modeling_finetune
import torch
from src.models.modeling_finetune import vit_base_patch16_160, vit_base_dim512_no_depth_patch16_160

# 定义模型
model1 = vit_base_patch16_160(pretrained=False)
model2 = vit_base_dim512_no_depth_patch16_160(pretrained=False)


# 用maedfer 进行一次前向

In [ ]:

# 创建随机输入数据
input_data = torch.rand(1, 3, 16, 160, 160)

# 进行前向传播
output1, output1_feature = model1(input_data,save_feature=True)
output2 = model2(input_data)

print(output1.shape)
print(output1_feature)
print(output2.shape)